<a href="https://colab.research.google.com/github/bnv20/cakd3/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)

Mounted at /content/drive


In [2]:
!pip install --target=$my_path "jpype1<1"

     |████████████████████████████████| 3.6 MB 5.4 MB/s 


In [3]:
!pip install --target=$my_path konlpy

     |████████████████████████████████| 19.4 MB 309 kB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 448 kB 76.5 MB/s 
     |████████████████████████████████| 15.7 MB 64.4 MB/s 
     |████████████████████████████████| 6.3 MB 61.8 MB/s 
     |████████████████████████████████| 62 kB 821 kB/s 
     |████████████████████████████████| 146 kB 72.6 MB/s 
     |████████████████████████████████| 145 kB 72.5 MB/s 
     |████████████████████████████████| 138 kB 77.8 MB/s 
     |████████████████████████████████| 59 kB 7.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.21.2 which is incompatible.
tensorflow 2.6.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.6.0 requires typing-extensions~=3.7.4, but you have typing-ex

In [ ]:
from google.colab import files
upload = files.upload()

In [5]:
!ls

drive  notebooks  sample_data


In [ ]:
!mv ratings_train.txt /content/drive/MyDrive/cakd3_colab/textmining/dataset

In [8]:
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/cakd3_colab/textmining/dataset/ratings_train.txt',sep='\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [9]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [11]:
train_df.document[1]

'흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'

In [14]:
import re

train_df = train_df.fillna(' ')
train_df['document'] = train_df['document'].apply(lambda x:re.sub(r"\d+"," ",x))

test_df = pd.read_csv('/content/drive/MyDrive/cakd3_colab/textmining/dataset/ratings_test.txt',sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x:re.sub(r"\d+", " ",x))

train_df.drop('id',axis=1, inplace=True)
test_df.drop('id',axis=1,inplace=True)


In [15]:
from konlpy.tag import Okt

okt = Okt()
def ok_tokenizer(text):
  tokens_ko = okt.morphs(text)
  return tokens_ko

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=ok_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])


/content/notebooks/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


[ 과제 ]

GridSearchCV를 이용하여 모델링 및 평가를 아래와 같이 수행하세요.
- 로지스틱 리그레션, 'C': [1 ,3.5, 4.5, 5.5, 10 ]
- cv=3 ,scoring='accuracy', verbose=1